In [1]:
%pip install pyodbc
%pip install duckdb
%pip install splink
%pip install usaddress
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyodbc
import os
import pandas as pd
import re
import usaddress
import time
import phonetics

/tmp/ipykernel_5470/2033548867.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
os.environ['DW_SERVER'] = "pdsqlirisdw01.hosted.lac.com"
os.environ['DW_DB'] = "PRD_APHIM_UODS_DW"
os.environ['DW_USER'] = "zz_dssteam"
os.environ['DW_PASS'] = "$3rv1se0neF1ve"

In [4]:
# Set up the connection
sqdw_con = pyodbc.connect(
    driver='sqlserver', 
    server=os.getenv('DW_SERVER'),
    database=os.getenv('DW_DB'),
    uid=os.getenv('DW_USER'),
    pwd=os.getenv('DW_PASS'),
    TrustServerCertificate='YES',
    timeout=10
)

#Execute a query 10k
query = "SELECT top 10000 * FROM Fact.VW_Person WITH (NOLOCK) where PER_StreetAddress is not NULL"
dw_person_10k = pd.read_sql(query, sqdw_con)
dw_person_10k["id"] = dw_person_10k.index + 1

/tmp/ipykernel_5470/2504593680.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dw_person_10k = pd.read_sql(query, sqdw_con)


In [5]:
#clean the data

In [6]:
input_df=dw_person_10k

In [7]:
%pip install pyodbc
%pip install duckdb
%pip install splink

#----
import pyodbc
import os
import pandas as pd
import re
import time
#----

start_time = time.time()
print(f"Execution time: {time.time() - start_time}")


input_df=input_df
#----
def clean_addr(df, address_var, apt_var, city_var, state_var, zip_var):
    # Replace backslashes with forward slashes in address
    df[address_var] = df[address_var].str.replace(r"\\", "/", regex=True)

    # Extract fractions like '1/2' or '3/4' from the address
    df['addr_frac'] = df[address_var].str.extract(r"(1/2|3/4)", expand=False)
    df[address_var] = df[address_var].str.replace(r"1/2|3/4", "", regex=True).str.strip()

    # Remove unnecessary punctuation and convert to uppercase
    df[address_var] = df[address_var].str.replace(r"[^a-zA-Z0-9\s,]", " ", regex=True).str.upper().str.strip()
    df[apt_var] = df[apt_var].str.replace(r"[^a-zA-Z0-9\s#\-/]", " ", regex=True).str.upper().str.strip()

    # Remove entirely blank entries
    for var in [address_var, apt_var, city_var, state_var, zip_var]:
        df[var] = df[var].replace(r"^\s*$", "", regex=True)

    # Simple regex to extract street (assuming 'number streetname' format)
    street_regex = r"^\d+\s[\w\s]+"
    df['addr'] = df[address_var].apply(lambda x: re.match(street_regex, x).group() if re.match(street_regex, x) else '')

    # Assuming city, state, and zip are already separated correctly
    df['city'] = df[city_var]
    df['state'] = df[state_var]
    df['zip'] = df[zip_var]

    return df

#----
# Now apply the clean_addr function to the input DataFrame
cleaned_df1=clean_addr(df=input_df,
                        address_var="PER_StreetAddress",
                        apt_var="PER_Apartment", 
                        city_var="PER_City", 
                        state_var="PER_State",
                        zip_var="PER_Zip")

#----
# Define the keyword lists
invalid_addr1 = "|".join(["UNKNO", "HOMELES", "NEED ADDR", "NEED INFO", "HOMESLESS", "TRANSIENT", "SUPPRESS", "ENCAMPM", 
                          "UNDOMIC", "UNDERPASS", "HOMELESS", "NOADDRESS", "NO ADDRESS", "NONE PROVIDED", "REFUSED", 
                          "123 DEFAULT ST", "PT STATES NONE", "999 TRANSIENT WAY", "UNSHELTERED", "UNHOUSED", "UNKKNOWN", 
                          "UNKNWOWN", "UNKONW", "UNKOWN", "UNSTABLE HOUSING", "XNEEDX", "XNEEDSX", "HOMELEXX", "X NEED X", 
                          "TRANSET", "MISSING", "MAILING ADDRESS", "MAILING ONLY", "NOT AVAILABLE", "NOT PROVIDED", 
                          "THIS IS NOT A HOME ADDRESS", "MAIL RTND BAD ADDRESS", "MAILING ADDRESS ONLY", "NEED NEW ADDRESS", 
                          "FILLING OUT FORMS", "ADDRS VERF", "NO ACTUAL ADDRESS", "NEEDS ADDRESS", "DOES NOT HAVE AN ADDRESS", 
                          "REQUEST CALL", "UNABLE TO PROVIDE", "RETURN MAIL", "BAD ADDRESS", "UNABLE TO LOCATE", 
                          "NOT GIVEN", "NONE GIVEN", "NO KNOWN ADDRESS"])
peh_keywords = invalid_addr1 + "|450 BAUCHET"
peh_keywords1 = ['NOT PROVIDED', 'UNSPECIFIED', '*NONE', 'REFUSED', '`', 'NULL', 'NONE', 'NA', 'N/A',  
                 'ODR', 'SHELTER', 'IN TRANSIT', 'UNK']
peh_keywords2 = peh_keywords1 + ['CA', 'NOT CALIFORNIA', 'LAC']

# Continue defining the keyword lists
invalid_addr15 = "|UPDATE DEMO MAIL RTN|123 TEST|TEST PATIENT|TEST STREET|UPDATE DEMO|ROUTINE POSTPARTUM|" + \
                "123 MISSING|^HOMELES.*|^UNHOUSED.*|^UNSHELTER.*|^TRANSIENT.*|^UNK.*|^PRIV.*|^UNSPEC.*|^PT STATES.*|"
invalid_addr3 = peh_keywords1 + ['YES', '1234 ST', 'TBA', 'TBD', 'TEST 1', 'TEST', 'UNITED STATES', 'US', 'USA',
                                 'MAIL ONLY', 'MAIL RETURNED', 'MAILING', 'ANTHEM BLUE CROSS', 'SANTA MONICA BEACH',
                                 'SPRING', 'TEXT', 'NAME', '1 HOME', 'HOTEL', '1 FAKE', 'SPANISH', 
                                 'SPECIMEN NOT COLLECTED',  'UPDATE', '1 NEED', 'MAIN HOUSE', 'MANAGER', 
                                 'MANAGER OFFICE', 'MANAGER UNIT', 'XX', 'XX OTHER', 'PATIENT REFUSED', 
                                 'NOT PROVIDED', "^REFUS.*", "^NO ADDRESS.*", 'NOT AVAILABLE']
invalid_addr4 = invalid_addr3 + ['CALIFORNIA', 'CA']
invalid_addr5 = invalid_addr4 + ['LOS ANGELES', 'LA', '# LA', '#LA', 'N A', 'NA', '999 TRANSIENT WAY', 
                                 '999 TRANSIENT', '99 JAIL CASE', '123 UNK', '9999 TRANSIENT', '9999 TRANSIENT WAY', 
                                 '9999 TRANSIENT', 'SOUTHBAY ER', 'XXX HOMELESS XXX', 'POST OFFICE BOX', 
                                 'PTIENT STATES', 'SUNSET ED', 'VALLEY PALMS CARE CENTER', 'W LOS ANGELES ED', 
                                 'WLA ED', 'UNK']

# Invalid apartment numbers
invalid_apt1 = invalid_addr1 + "|UNITED CARE FACILITES|HEALTH CARE|WE DO HAVE A NEW REGISTRATION|" + \
               "UNITED CEREBRAL PALSEY|UNITED STATES"
invalid_apt2 = invalid_addr4 + ['YES', '-', '1 LAB', 'APT SUITE FLOOR ETC.', 'NO APT']
invalid_apt3 = invalid_addr5 + ["UNITED CARE FACILITES", "HEALTH CARE", "WE DO HAVE A NEW REGISTRATION", 
                                "UNITED CEREBRAL PALSEY", "UNITED STATES", "BELLFLOWER CHRISTIAN", 
                                "GRAND PARK CONVALESCENT", 'BRUIN PLAZA', 'YES', '-', '1 LAB', 
                                'APT SUITE FLOOR ETC', 'NO APT']

# Key phrases for invalid apartment units
invalid_apt_frag = ["CONV", "HEALTH", "MEDICAL", "NURSING", "ACUTE", "CARE", "SOBER", "REHAB", "ASSISTED",
                    "LIVING", "MANOR", "VILLA", "RETIREMENT", "BEH", "SENIOR", "SNF", "RESID", "SANIT", "CHURCH",
                    "ACUTE", "HEALTHCARE", "WELLNESS", "CONGREGATE", "CONVALESCENT", "GLENDORA GRAND", "REHABILITATION",
                    # Other terms
                    "HOME", "HOUSE", 'HEIGHT', 'COUNTRY', 'GARDEN', 'COTTAGE',  'ADDR', 'C/O', 
                    'POBOX', 'PLAZA', 'UNABLE', 'TRANSIENT', 'JAIL', 'PRISON', 'THE', 'SAME', 'VISTA', 'MAIL', 'PARK', 
                    'OFFICE', 'LAPD', 'LETTER', 'HOTEL', 'INN', 'MOTEL', 'COUNTY', 'UPPER', 'LOWER', 'REAR', 'DOWN', 
                    'UP', 'TEST', 'UNI', 'STADIUM']

# Invalid cities
invalid_city = ['NULL', 'NONE', 'NA', 'N/A', 'YES', 'REFUSED', '`', 'NOT PROVIDED', 'UNK', '', 'UNKKNOWN', 
                'UNKNWOWN', 'UNKONW', 'UNKOWN', 'UNSPECIFIED', 'UNITED STATES', 'US', 'CALIFORNIA', 'CA', ' ', '*NONE']

# POBOX keywords
pobox_keywords = "POBOX|PO BOX|P0 BOX|POSTAL|PO BX|9O BOX|OK BOX|P 0  BOX|P BOX|PB BOX|PI BOX|PIC BOX|P.O. BOX|" + \
                 "P.O.BOX|PIO BOX|PITZER BOX|PO B0X|PO BAX|PO BOS|PO BOT|PO BOV|POX BOX|SHOW BOX|POP BOX|P O BOX|POX BOX"

# Unknown phone numbers
unknown_numbers = ["0000000000", "1111111111", "2222222222", "3333333333", "4444444444", "5555555555", 
                   "6666666666", "7777777777", "8888888888", "9999999999", "0000000001"]

# Unknown email keywords
unknown_emails1 = "GENERATED|NOEMAIL|NOMAIL|REFUSE|UNKNOWN|NO@NO|NA@NA|PATIENT|" + \
                  "DOES NOT|MISSING|NONAME|DECLINED|ASK FOR|NOT GIVEN"
unknown_emails2 = ['NAGMAIL.COM', 'N', 'DECLINED', 'NA', 'NO', 'TEST', 'NONE', 'GOBACKTOASK']

# Unknown names
unknown_names1 = "NO FIRST NAME|NO INFO"
unknown_names2 = ["TEST", "PATIENT"]
#----

# Assuming invalid_addr1, invalid_addr2, ..., invalid_apt1, ..., etc. are lists
# Convert any string components to lists
invalid_addr1_list=invalid_addr1.split('|')
invalid_addr15_list=invalid_addr15.split('|')
pobox_keywords_list = pobox_keywords.split('|')
invalid_apt1_list = invalid_apt1.split('|')
unknown_emails1_list = unknown_emails1.split('|')
unknown_names1_list = unknown_names1.split('|')

# Combine all keywords into a single list
#all_invalid_keywords = invalid_addr1_list + invalid_addr15_list +invalid_addr3 + invalid_addr4 +\
#invalid_addr5 + invalid_apt1_list + invalid_apt2 + invalid_apt3 + pobox_keywords_list + unknown_numbers +\
#unknown_emails1_list +  unknown_emails2 + unknown_names1_list + unknown_names2

# Combine all keywords into a single list
# Ensure all keywords are strings and in uppercase for case-insensitive comparison
all_invalid_keywords = [str(keyword).upper() for keyword in ( invalid_addr1_list + invalid_addr15_list +invalid_addr3 + invalid_addr4 +\
invalid_addr5 + invalid_apt1_list + invalid_apt2 + invalid_apt3 + pobox_keywords_list + unknown_numbers +\
unknown_emails1_list +  unknown_emails2 + unknown_names1_list + unknown_names2)]

def check_invalid_component(parsed_addr, invalid_keywords):
    for component_type, component_value in parsed_addr.items():
        # Skip certain components from direct keyword matching
        if component_type in ['AddressNumber', 'AddressNumberPrefix', 'AddressNumberSuffix', 'StreetName', 'StreetNamePostType', 
                    'StreetNamePreDirectional','OccupancyType','OccupancyIdentifier','PlaceName','StreetNamePreType']:
            continue

        if any(keyword in component_value.upper() for keyword in invalid_keywords):
            print(f"Invalid component found: {component_type} -> {component_value}")
            return True
    return False

def clean_address(address, invalid_keywords):
    try:
        parsed_address, _ = usaddress.tag(address)
        if check_invalid_component(parsed_address, invalid_keywords):
            return ""  # Return empty string if invalid
        return address  # Return original address if valid
    except usaddress.RepeatedLabelError as e:
        print(f"Error parsing address '{address}': {e}")
        return ""

input_df2 = cleaned_df1



# Apply the cleaning function to the 'addr' field
input_df2['addr_valid'] = input_df2['addr'].apply(lambda x: clean_address(x, all_invalid_keywords))

#----
#Here we will do the same thing with the city, state and zipcodes i.e. compare them to the invalid keywords

# Define invalid keyword lists for city, state, and zipcode
invalid_city_keywords = invalid_addr4 # Add more as needed
invalid_state_keywords = invalid_addr3  # Add more as needed
invalid_zip_keywords = invalid_addr5  # Add more as needed

def check_invalid_component(parsed_addr, city_keywords, state_keywords, zip_keywords):
    is_invalid = False
    for component_type, component_value in parsed_addr.items():
        if component_type == 'PlaceName' and any(kw in component_value.upper() for kw in city_keywords):
            is_invalid = True
        elif component_type == 'StateName' and any(kw in component_value.upper() for kw in state_keywords):
            is_invalid = True
        elif component_type == 'ZipCode' and any(kw in component_value.upper() for kw in zip_keywords):  # Corrected key
            is_invalid = True
    return is_invalid

def clean_location(city, state, zipcode, city_keywords, state_keywords, zip_keywords):
    try:
        pseudo_address = f"{city}, {state} {zipcode}"  # Corrected variable name
        parsed_address, _ = usaddress.tag(pseudo_address)
        if check_invalid_component(parsed_address, city_keywords, state_keywords, zip_keywords):
            return "", "", ""  # Return empty strings if invalid
        return city, state, zipcode  # Return original values if valid
    except usaddress.RepeatedLabelError:
        print(f"Error parsing pseudo address '{city}, {state} {zipcode}'")
        return "", "", ""  # Handle parsing errors

# Apply the function (assuming input_df2 is defined)
input_df2[['city_valid', 'state_valid', 'zip_valid']] = input_df2.apply(
    lambda row: clean_location(row['city'], row['state'], row['zip'],
                               invalid_city_keywords, invalid_state_keywords, invalid_zip_keywords),
    axis=1, result_type='expand')

#----
#I need to also do the same thing for the apt field
# Define your list of invalid keywords for apartment numbers
invalid_apt_keywords = invalid_addr5  # Add your keywords

def is_valid_apt(apt, invalid_keywords):
    # Check if the apartment number is in the invalid keywords
    if any(kw.lower() in apt.lower() for kw in invalid_keywords):
        return ""
    return apt

# Apply the function to the 'PER_Apartment' field
input_df2['apt_valid'] = input_df2['PER_Apartment'].apply(lambda x: is_valid_apt(x, invalid_apt_keywords))

#----
apt_keywords = ['Apt', 'Unit', 'Suite', 'Bldg', 'Room', 'Fl', '#']

# Regular expression pattern for extracting apartment/unit numbers
apt_pattern = r'\b(?:' + '|'.join(apt_keywords) + r')\s*\S+'

def extract_apt(addr):
    match = re.search(apt_pattern, addr, re.IGNORECASE)
    return match.group(0) if match else ""

# Extract and populate the apt_valid field
input_df2['apt_valid'] = input_df2['addr_valid'].apply(extract_apt)
#----
#I am sure that there is an easier way of doing what I did above but it seems to work the way Tinas code worked
#----

def clean_repeat_char(df, var, repeat_num=0):
    # Create a regular expression for repeated characters
    repeat_char_regex = r"^(.)\1*$"

    # Identify rows with invalid data
    df['invalid'] = df[var].apply(lambda x: (bool(re.match(repeat_char_regex, x)) and len(x) > repeat_num) or x in ["0", "00"])

    # Replace invalid data with an empty string
    df[var + '_new'] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Assuming cleaned_df2 is your DataFrame and it already has an 'apt_valid' column
# If not, you need to create it before applying the function
# For example:
# cleaned_df2['apt_valid'] = ...

cleaned_df3 = input_df2
cleaned_df3 = clean_repeat_char(input_df2, "addr_valid")
cleaned_df3 = clean_repeat_char(cleaned_df3, "apt_valid", repeat_num=3)
cleaned_df3 = clean_repeat_char(cleaned_df3, "city_valid")
cleaned_df3 = clean_repeat_char(cleaned_df3, "state_valid")
cleaned_df3 = clean_repeat_char(cleaned_df3, "zip_valid")

# Update original columns with the cleaned data
cleaned_df3['addr_valid'] = cleaned_df3['addr_valid_new']
cleaned_df3['apt_valid'] = cleaned_df3['apt_valid_new']
cleaned_df3['city_valid'] = cleaned_df3['city_valid_new']
cleaned_df3['state_valid'] = cleaned_df3['state_valid_new']
cleaned_df3['zip_valid'] = cleaned_df3['zip_valid_new']

# Drop the temporary columns
cleaned_df3.drop(columns=['invalid', 'addr_valid_new', 'apt_valid_new', 'city_valid_new', 'state_valid_new', 'zip_valid_new'], inplace=True)

#----
def clean_spec_char(df, var):
    # Define a regex pattern that checks for the absence of alphanumeric characters
    spec_char_regex = r"^[^a-zA-Z\d]+$"

    # Identify rows with only special characters and mark them as invalid
    df['invalid'] = df[var].apply(lambda x: bool(re.match(spec_char_regex, x)) if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Apply the function to the DataFrame
# Assume cleaned_df3 is your existing DataFrame
cleaned_df4 = clean_spec_char(cleaned_df3, "addr_valid")
cleaned_df4['addr_valid'] = cleaned_df4['new_addr_valid']
cleaned_df4 = clean_spec_char(cleaned_df4, "apt_valid")
cleaned_df4['apt_valid'] = cleaned_df4['new_apt_valid']
cleaned_df4 = clean_spec_char(cleaned_df4, "city_valid")
cleaned_df4['city_valid'] = cleaned_df4['new_city_valid']
cleaned_df4 = clean_spec_char(cleaned_df4, "state_valid")
cleaned_df4['state_valid'] = cleaned_df4['new_state_valid']
cleaned_df4 = clean_spec_char(cleaned_df4, "zip_valid")
cleaned_df4['zip_valid'] = cleaned_df4['new_zip_valid']

# Drop the temporary columns
cleaned_df4.drop(columns=['invalid', 'new_addr_valid', 'new_apt_valid', 'new_city_valid', 'new_state_valid', 'new_zip_valid'], inplace=True)

#----
def clean_num_only(df, var):
    # Remove all non-alphanumeric characters and check if the remaining string is numeric
    df['invalid'] = df[var].apply(lambda x: x.isalnum() and x.isdigit() if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Apply the function to the DataFrame
# Assume cleaned_df4 is your existing DataFrame
cleaned_df5 = clean_num_only(cleaned_df4, "addr_valid")
cleaned_df5['addr_valid'] = cleaned_df5['new_addr_valid']
cleaned_df5 = clean_num_only(cleaned_df5, "city_valid")
cleaned_df5['city_valid'] = cleaned_df5['new_city_valid']
cleaned_df5 = clean_num_only(cleaned_df5, "state_valid")
cleaned_df5['state_valid'] = cleaned_df5['new_state_valid']

# Drop the temporary columns
cleaned_df5.drop(columns=['invalid', 'new_addr_valid', 'new_city_valid', 'new_state_valid'], inplace=True)
cleaned_df5
#----

def remove_email(df, var):
    # Define a regex pattern for email addresses
    email_regex = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"

    # Identify rows with email addresses and mark them as invalid
    df['invalid'] = df[var].apply(lambda x: bool(re.search(email_regex, x)) if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Example usage
# Assume you have a DataFrame named 'your_df' with a column 'original_address'
# your_df = pd.DataFrame(...) # your actual DataFrame
cleaned_df6 = remove_email(cleaned_df5, "PER_Email")
#cleaned_df6['original_address'] = cleaned_df6['new_original_address']
#cleaned_df.drop(columns=['invalid', 'new_original_address'], inplace=True)

#----
def clean_dates_only(df, var):
    # Define regex patterns for date formats
    date_patterns = [
        r"\d{2}/\d{2}/\d{4}",  # mm/dd/yyyy
        r"\d{2}/\d{2}/\d{2}",  # mm/dd/yy
        r"\d{2}-\d{2}-\d{4}",  # mm-dd-yyyy
        r"\d{2}-\d{2}-\d{2}"   # mm-dd-yy
    ]

    # Combine patterns into a single regex
    combined_pattern = '|'.join(date_patterns)

    # Identify rows with date-like strings
    df['invalid'] = df[var].apply(lambda x: bool(re.search(combined_pattern, x)) if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Apply the function to the DataFrame
# Assume cleaned_df6 is your existing DataFrame
cleaned_df7 = clean_dates_only(cleaned_df6, "addr_valid") ## we only limited it to the address components need to use df5 instead of df6
cleaned_df7['addr_valid'] = cleaned_df7['new_addr_valid']
cleaned_df7 = clean_dates_only(cleaned_df7, "apt_valid")
cleaned_df7['apt_valid'] = cleaned_df7['new_apt_valid']
cleaned_df7 = clean_dates_only(cleaned_df7, "city_valid")
cleaned_df7['city_valid'] = cleaned_df7['new_city_valid']
cleaned_df7 = clean_dates_only(cleaned_df7, "state_valid")
cleaned_df7['state_valid'] = cleaned_df7['new_state_valid']
cleaned_df7 = clean_dates_only(cleaned_df7, "zip_valid")
cleaned_df7['zip_valid'] = cleaned_df7['new_zip_valid']

# Drop the temporary columns
cleaned_df7.drop(columns=['invalid', 'new_addr_valid', 'new_apt_valid', 'new_city_valid', 'new_state_valid', 'new_zip_valid'], inplace=True)

#----
def clean_two_word(df, var):
    street_suffix = ["RD", "ST", "ROAD", "STREET", "PL", "PLACE", "LN", "LANE", "UPDATE", "OTHER"]
    
    def is_invalid(address):
        if pd.isna(address):
            return False
        words = re.sub(r"[^a-zA-Z0-9\s]", "", address.replace("  ", " ")).split()
        if len(words) != 2:
            return False
        return (words[1] in street_suffix and 
                bool(re.match(r"^(.)\1*$", words[0])) and 
                len(words[0]) > 1) or (words[0].replace("0", "") == "")

    df['invalid'] = df[var].apply(is_invalid)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df
#----

# Apply the function to the DataFrame
# Assume cleaned_df7 is your existing DataFrame
cleaned_df8 = clean_two_word(cleaned_df7, "addr_valid")
cleaned_df8['addr_valid'] = cleaned_df8['new_addr_valid']
cleaned_df8 = clean_two_word(cleaned_df8, "apt_valid")
cleaned_df8['apt_valid'] = cleaned_df8['new_apt_valid']
cleaned_df8 = clean_two_word(cleaned_df8, "city_valid")
cleaned_df8['city_valid'] = cleaned_df8['new_city_valid']
cleaned_df8 = clean_two_word(cleaned_df8, "state_valid")
cleaned_df8['state_valid'] = cleaned_df8['new_state_valid']
cleaned_df8 = clean_two_word(cleaned_df8, "zip_valid")
cleaned_df8['zip_valid'] = cleaned_df8['new_zip_valid']

# Drop the temporary columns
cleaned_df8.drop(columns=['invalid', 'new_addr_valid', 'new_apt_valid', 'new_city_valid', 'new_state_valid', 'new_zip_valid'], inplace=True)

#----
def unit_parse(df, var):
    # List of apartment keywords
    apt_names_st = ['APT', 'APP', 'AP', 'APACE', 'APARTMENT', 'CONDO', 'NO', 'STE', 'SUITE', 'SPC', 'UNIT', 
                    'BUILD', 'BLDG', 'BUILDING', 'RM', 'ROOM']

    # Create regex pattern for extracting unit
    apt_pattern_out = r"(?i)\b(" + "|".join(apt_names_st) + r")\s*\#?\s*[\d\w]+"

    # Function to extract unit
    def extract_unit(address):
        match = re.search(apt_pattern_out, address)
        return match.group(0) if match else ''

    # Function to remove unit
    def remove_unit(address):
        return re.sub(apt_pattern_out, '', address).strip()

    # Apply the functions to create new columns
    df['new_address'] = df[var].apply(remove_unit)
    df['new_address_unit'] = df[var].apply(extract_unit)

    return df


cleaned_df9 = unit_parse(cleaned_df8, "addr_valid")

print(f"Execution time: {time.time() - start_time}")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Execution time: 0.00011014938354492188
Invalid component found: StreetNamePostDirectional -> W
Invalid component found: StreetNamePostDirectional -> N
Invalid component found: StreetNamePostDirectional -> E
Invalid component found: StreetNamePostDirectional -> E
Invalid component found: StreetNamePostDirectional -> W
Invalid component found: StreetNamePostDirectional -> W
Invalid component found: StreetNamePostDirectional -> W
Invalid component found: StreetNamePostDirectional -> W
Invalid component found: StreetNamePostDirectional -> W
Invalid component found: StreetNamePostDirectional -> E
Invalid component found: StreetNamePostDirectional -> W
Invalid component found: StreetNamePostDirectional -> E
Invalid component found: StreetNamePostDirectional -> E
Invalid component found: StreetName

In [8]:
#The blockers that Tina uses are:fname, mname, lname, dob, phn1, phn2, email, ssn, and addr_var

#'PER_LastName', 'PER_FirstName', 'PER_MiddleName',  'PER_SSN', 'PER_DOB',  'PER_HomePhone', 'PER_CellularPhoneOrPager',
#        'PER_Email'
    
cleaned_df9_subset=cleaned_df9[['PER_LastName', 'PER_FirstName', 'PER_MiddleName',  'PER_SSN', 'PER_DOB',  'PER_HomePhone', 'PER_CellularPhoneOrPager',
        'PER_Email', 'city_valid', 'state_valid',
       'zip_valid', 'apt_valid', 'new_address', 'new_address_unit', 'id']]

In [9]:
#Feature engineering for RecordLinkage/Deduplication

import pandas as pd
import phonetics

# Define a function to apply the dmetaphone phonetic algorithm to each name in the column
def dmetaphone_name(name):
    if name is None:
        pass
    else:
        return phonetics.dmetaphone(name)

# Apply the function to the "first_name", lastname and middlename columns using the apply method
cleaned_df9_subset['firstname_dm'] = cleaned_df9_subset['PER_FirstName'].apply(dmetaphone_name)
cleaned_df9_subset['lastname_dm'] = cleaned_df9_subset['PER_LastName'].apply(dmetaphone_name)
cleaned_df9_subset['middlename_dm'] = cleaned_df9_subset['PER_MiddleName'].apply(dmetaphone_name)


# Create a new column "full_name" by combining "first_name" and "surname" columns
cleaned_df9_subset['full_name'] = cleaned_df9_subset['PER_FirstName'] + ' ' + cleaned_df9_subset['PER_LastName']



/tmp/ipykernel_5470/1784819825.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df9_subset['firstname_dm'] = cleaned_df9_subset['PER_FirstName'].apply(dmetaphone_name)
/tmp/ipykernel_5470/1784819825.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df9_subset['lastname_dm'] = cleaned_df9_subset['PER_LastName'].apply(dmetaphone_name)
/tmp/ipykernel_5470/1784819825.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [10]:
counts = cleaned_df9_subset.isna().sum()
print(counts.sort_values())

PER_LastName                   0
lastname_dm                    0
firstname_dm                   0
id                             0
new_address_unit               0
new_address                    0
apt_valid                      0
zip_valid                      0
state_valid                    0
PER_SSN                        0
PER_FirstName                  0
city_valid                     0
full_name                      0
PER_DOB                      199
PER_HomePhone               1831
middlename_dm               6402
PER_MiddleName              6402
PER_Email                   7854
PER_CellularPhoneOrPager    7854
dtype: int64


In [12]:
from splink.duckdb.linker import DuckDBLinker
#from splink.spark.linker import SparkLinker

settings = {
    "unique_id_column_name": "id",
    "link_type": "dedupe_only",
}

linker = DuckDBLinker(cleaned_df9_subset, settings)

In [13]:
from splink.duckdb.blocking_rule_library import block_on

blocking_rules = [
        block_on("lastname_dm"),
        block_on("PER_LastName"),
        block_on("new_address"),
        block_on("full_name"),
        block_on("firstname_dm"),
        block_on("PER_FirstName"),
        block_on("PER_DOB"),
        block_on("zip_valid"),
        block_on("PER_HomePhone"),
        block_on("PER_Email"),
        block_on("PER_CellularPhoneOrPager"),
        block_on("middlename_dm"),
        block_on("PER_SSN")
]
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

alt.Chart(...)

In [15]:
import splink.duckdb.comparison_template_library as ctl
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl


import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl


settings = {
    "unique_id_column_name": "id",
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": blocking_rules,
    "comparisons": [
        ctl.name_comparison("lastname_dm"),
        ctl.name_comparison("firstname_dm"),
        ctl.name_comparison("PER_FirstName"),
        ctl.name_comparison("PER_LastName"),
        ctl.name_comparison("full_name"),
        ctl.email_comparison("PER_Email", include_username_fuzzy_level=False),
        ctl.forename_surname_comparison("PER_FirstName","PER_LastName"),
        cl.levenshtein_at_thresholds("PER_SSN", [2]),
        cl.exact_match("new_address", term_frequency_adjustments=True),
        ctl.date_comparison("PER_DOB"),
    ],
    "retain_intermediate_calculation_columns": True
}

linker = DuckDBLinker(cleaned_df9_subset, settings, set_up_basic_logging=False)
deterministic_rules = [
    "l.PER_SSN = r.PER_SSN and l.PER_DOB = r.PER_DOB",
    "l.PER_FirstName = r.PER_FirstName and l.PER_DOB = r.PER_DOB",
    "l.PER_LastName = r.PER_LastName and l.PER_DOB = r.PER_DOB",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

Probability two random records match is estimated to be  5.71e-05.
This means that amongst all possible pairwise record comparisons, one in 17,521.61 are expected to match.  With 49,995,000 total possible comparisons, we expect a total of around 2,853.33 matching pairs


In [16]:
linker.estimate_u_using_random_sampling(max_pairs=1e8)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - lastname_dm (no m values are trained).
    - firstname_dm (no m values are trained).
    - PER_FirstName (no m values are trained).
    - PER_LastName (no m values are trained).
    - full_name (no m values are trained).
    - PER_Email (no m values are trained).
    - custom_PER_LastName_PER_FirstName (no m values are trained).
    - PER_SSN (no m values are trained).
    - new_address (no m values are trained).
    - PER_DOB (no m values are trained).


In [17]:
linker.estimate_m_from_label_column("PER_SSN")

-------- Estimating m probabilities using from column PER_SSN --------
m probability not trained for PER_SSN - Levenshtein <= 2 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
m probability not trained for PER_SSN - All other comparisons (comparison vector value: 0). This usually means the comparison level was never observed in the training data.

Your model is not yet fully trained. Missing estimates for:
    - PER_SSN (some m values are not trained).


In [18]:
linker.estimate_m_from_label_column("PER_DOB")

-------- Estimating m probabilities using from column PER_DOB --------
m probability not trained for PER_Email - Exact match PER_Email (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
m probability not trained for PER_Email - Jaro_winkler_similarity PER_Email >= 0.88 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
m probability not trained for PER_DOB - Damerau_levenshtein <= 1 (comparison vector value: 4). This usually means the comparison level was never observed in the training data.
m probability not trained for PER_DOB - Within 1 month (comparison vector value: 3). This usually means the comparison level was never observed in the training data.
m probability not trained for PER_DOB - Within 1 year (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
m probability not trained for PER_DOB - Within 10 ye

In [19]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [20]:
linker.m_u_parameters_chart()

alt.HConcatChart(...)

In [21]:
linker.unlinkables_chart()

#note this result doesnt make sense to me

alt.LayerChart(...)

In [23]:
df_predictions = linker.predict(threshold_match_probability=0.2)
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predictions, threshold_match_probability=0.5)
df_predictions.as_pandas_dataframe()
df_predictions

Completed iteration 1, root rows count 6154
Completed iteration 2, root rows count 5537
Completed iteration 3, root rows count 5186
Completed iteration 4, root rows count 3392
Completed iteration 5, root rows count 1953
Completed iteration 6, root rows count 590
Completed iteration 7, root rows count 189
Completed iteration 8, root rows count 35
Completed iteration 9, root rows count 3
Completed iteration 10, root rows count 0


Table name in database: `__splink__df_predict_2c8c14497`

To retrieve records, you can call the following methods on this object:
`.as_record_dict(limit=5)` or `.as_pandas_dataframe(limit=5)`.

You may omit the `limit` argument to return all records.

This table represents the following splink entity: __splink__df_predict

In [24]:
#Waterfall chart¶ The waterfall chart provides a means of visualising individual 
#predictions to understand how Splink computed the final matchweight for a particular pairwise 
#record comparison.

records_to_view  = df_predictions.as_record_dict(limit=5)
linker.waterfall_chart(records_to_view, filter_nulls=False)

alt.LayerChart(...)

In [25]:
#The comparison viewer dashboard takes this one step further by producing an interactive dashboard 
#that contains example predictions across the spectrum of match scores.

linker.comparison_viewer_dashboard(df_predictions, "scv.html", overwrite=True)

# You can view the scv.html file in your browser, or inline in a notbook as follows
from IPython.display import IFrame
IFrame(
    src="./scv.html", width="100%", height=1200
)  